In [1]:
import pandas as pd
import numpy as np
import h5py
from tflearn.data_utils import build_hdf5_image_dataset
import h5py
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import tflearn
from tflearn.layers.normalization import local_response_normalization

Building the train dataset

####Build the dataset (only required once)
id_train = pd.read_csv('../Data/id_train.csv')#replace by path of the id_train file 
file = pd.DataFrame(id_train['Id'].apply(lambda x: './train_imgs/' + str(x) + '.jpg' ))#Build the path to each image
file['label'] = id_train['label'] - 1 #Label goes from 1 to 4, we want 0 to 3 
file.to_csv('file_train.csv', sep = ' ', header = False, index=False)
dataset_file = 'file_train.csv'
build_hdf5_image_dataset(dataset_file, image_shape=(28, 28), mode='file', output_path='dataset.h5', categorical_labels=True, normalize=True)

Loading the Dataset

In [2]:
#Loading the dataset
h5f = h5py.File('train32_dataset.h5', 'r')
X  = h5f['X']
Y = h5f['Y']

In [3]:
# Shuffle the data
X, Y = shuffle(X, Y)

In [4]:
X.shape

(8000, 32, 32, 3)

In [5]:
Y.shape

(8000, 4)

In [6]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center(mean=0.45741)
img_prep.add_featurewise_stdnorm(std=0.228904)

In [7]:
# Create extra synthetic training data by flipping, rotating and blurring the images on our data set.
img_aug = ImageAugmentation()
#img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)


Building the network

In [8]:
network = input_data(shape=[None, 32, 32, 3], data_preprocessing=img_prep, data_augmentation=img_aug)
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 4)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 4)
network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 64, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 4, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy')

In [9]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0)

Training the model

In [10]:
epochs = 1000

In [12]:
model.fit(X, Y, n_epoch=50, shuffle=True,
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
run_id='roof-classifier', validation_set=0.2)
model.save("roof-classifier_v3.tfl")
print("Network trained and saved as roof-classifier_v3.tfl!")

Training Step: 70350  | total loss: 0.91213
| Adam | epoch: 050 | loss: 0.91213 - acc: 0.6271 | val_loss: 0.90591 - val_acc: 0.6425 -- iter: 6400/6400
Training Step: 70350  | total loss: 0.91213
| Adam | epoch: 050 | loss: 0.91213 - acc: 0.6271 | val_loss: 0.90591 - val_acc: 0.6425 -- iter: 6400/6400
--
Network trained and saved as roof-classifier_v3.tfl!


In [15]:
hhh=h5py.File('test32_dataset.h5', 'r')

In [19]:
hhh["X"]

<HDF5 dataset "X": shape (13999, 32, 32, 3), type "<f4">